In [ ]:
# processes drought indicator(s): total water storage anomaly

In [1]:
import os
import sys
import math
import warnings
import geopandas as gpd
from osgeo import gdal
from pathlib import Path
import glob
import numpy as np
import rasterio.mask
import rasterio
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [2]:
country = os.getcwd().split('\\')[-1]

In [3]:
shp = gpd.read_file('shapefile/' + country.replace(' ', '_').lower() + '.shp').buffer(50)
features = shp.geometry

C:\Users\Owner\AppData\Local\Temp\ipykernel_13256\1376300571.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp = gpd.read_file('shapefile/' + country.replace(' ', '_').lower() + '.shp').buffer(50)


In [4]:
# Raw data folder. Change file path as needed
folder_list = [x[0] for x in os.walk(r'F:\World Bank\City Scan\data\Drought')]
folder_list.remove(folder_list[0])

In [5]:
raster_list = {#'rdria': 't',
               #'spg01': 'm',
               #'spg12': 'm',
               'twsan': 'm'}

In [6]:
month_list = ['01', '02', '03', '04', '05', '06',
              '07', '08', '09', '10', '11', '12']

In [7]:
def clip_drought(folder, raster_prefix, raster_suffix, year, date):    
    try:
        with rasterio.open(folder + '/' + raster_prefix + '_m_wld_' + year + date + '01_' + raster_suffix + '.tif') as src:
            out_image, out_transform = rasterio.mask.mask(
                src, features, crop=True)
            out_meta = src.meta.copy()

        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        with rasterio.open('output/drought/' + raster_prefix + '_' + year + date + ".tif", "w", **out_meta) as dest:
            dest.write(out_image)
    
    except:
        pass

In [8]:
for raster in raster_list.keys():
    for year in range(2011, 2021):
        for month in month_list:
            clip_drought(folder_list[3], raster, raster_list.get(raster), str(year), str(month))